In [27]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import matplotlib.pyplot as plt
import h5py
import wandb
import random


# Configure Keras to use GPU
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)



2024-04-01 14:37:11.396640: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21607 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6


In [4]:
import wandb
import tensorflow as tf
import numpy as np
import h5py
from tensorflow.keras import layers, models, optimizers

# Ensure wandb is installed: pip install wandb
# Initialize your W&B project before running the script

# Dataset loading and preparation
def load_and_prepare_dataset(batch_size):
    with h5py.File('TrainingData5zeroes.h5', 'r') as hdf:
        images = np.array(hdf.get('images'))
        boxes = np.array(hdf.get('boxes'))
    

    image_normalized = (images + 1e-9) / 9.26
    normalized_boxes = boxes / [1, 64, 64, 64, 64]

    images_np = image_normalized
    probabilities = np.array(normalized_boxes[:, :, :-4])
    probabilities = tf.expand_dims(probabilities, axis=1)
    boxes_np = np.array(normalized_boxes[:, :, 1:])
    boxes_np = tf.expand_dims(boxes_np, axis=1)

    dataset = tf.data.Dataset.from_tensor_slices((images_np, {'x_prob_reshape': probabilities, 'x_boxes_reshape': boxes_np}))
    dataset = dataset.shuffle(buffer_size=8000).batch(batch_size)
    return dataset

# Model building
def build_model():
    input_shape = (64, 64, 1)
    num_classes = 280
    num_coordinates = 4

    x_input = layers.Input(shape=input_shape)
    #Layer 1
    x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x_input)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x) 
    
    x = layers.Conv2D(64, kernel_size=3, padding='same', activation='relu')(x)
    #Layer 2
    x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.Conv2D(128, kernel_size=3, padding='same', activation='relu')(x)
    #Layer 3
    x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, kernel_size=3, padding='same', activation='relu')(x)
    #Layer 4
    x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.Conv2D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    #Layer 5
    x = layers.Conv2D(256, kernel_size=5, padding='same', activation='relu')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x) 
    
    
    x = layers.Flatten()(x)
    x_prob = layers.Dense(num_classes, activation='sigmoid', name='x_prob')(x)
    x_boxes = layers.Dense(num_classes * num_coordinates, activation='sigmoid', name='x_boxes')(x)
    x_prob_reshape = layers.Reshape((-1, num_classes, 1), name='x_prob_reshape')(x_prob)
    x_boxes_reshape = layers.Reshape((-1, num_classes, num_coordinates), name='x_boxes_reshape')(x_boxes)

    model = models.Model(x_input, [x_prob_reshape, x_boxes_reshape])
    return model

# Optimizer selection
def select_optimizer(optimizer_name, learning_rate):
    if optimizer_name == 'adam':
        return optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == 'sgd':
        return optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        raise ValueError("Unsupported optimizer specified.")

# Training function with sweep
def train_with_sweep():
    
    with wandb.init():
        config = wandb.config

        dataset = load_and_prepare_dataset(config.batch_size)
        model = build_model()
        # model = tf.keras.models.load_model('M11overfittedmodel3variant.h5')
        optimizer = select_optimizer(config.optimizer, config.learning_rate)
        model.compile(optimizer=optimizer, 
                      loss={'x_prob_reshape': 'binary_crossentropy', 'x_boxes_reshape': 'mean_squared_error'}, 
                      metrics={'x_prob_reshape': 'accuracy'})

        # Fit the model
        model.fit(dataset, epochs=config.epochs, callbacks=[wandb.keras.WandbCallback()])
        


In [5]:

sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        }
    # 'fc_layer_size': {
    #     'values': [128, 256, 512]
    #     },
    # 'dropout': {
    #       'values': [0.3, 0.4, 0.5]
    #     },
    }

sweep_config['parameters'] = parameters_dict
parameters_dict.update({
    'epochs': {
        'value': 1000}
    })
parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        'values': [32, 64, 256]
      }
    })

In [6]:
sweep_config


{'method': 'random',
 'metric': {'name': 'loss', 'goal': 'minimize'},
 'parameters': {'optimizer': {'values': ['adam', 'sgd']},
  'epochs': {'value': 1000},
  'learning_rate': {'distribution': 'uniform', 'min': 0, 'max': 0.1},
  'batch_size': {'values': [32, 64, 256]}}}

In [7]:
sweep_id = wandb.sweep(sweep_config, project="Working Hyperparameter Tuning")
wandb.agent(sweep_id, train_with_sweep, count=5)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: hvu4lu3b
Sweep URL: https://wandb.ai/alphanium/Working%20Hyperparameter%20Tuning/sweeps/hvu4lu3b


wandb: Agent Starting Run: 1106kk3r with config:
wandb: 	batch_size: 64
wandb: 	epochs: 1000
wandb: 	learning_rate: 0.019313268057660163
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: derrickappiahosei (alphanium). Use `wandb login --relogin` to force relogin


2024-04-01 14:48:01.688478: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21940 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:68:00.0, compute capability: 8.6
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/1000


...

Epoch 1/1000
40/40 [==============================] - 6s 121ms/step - loss: 0.4512 - x_prob_reshape_loss: 0.3102 - x_boxes_reshape_loss: 0.1409 - x_prob_reshape_accuracy: 0.8764
Epoch 2/1000
40/40 [==============================] - 5s 121ms/step - loss: 0.1975 - x_prob_reshape_loss: 0.1308 - x_boxes_reshape_loss: 0.0667 - x_prob_reshape_accuracy: 0.9526
Epoch 3/1000
40/40 [==============================] - 5s 121ms/step - loss: 0.1600 - x_prob_reshape_loss: 0.1046 - x_boxes_reshape_loss: 0.0554 - x_prob_reshape_accuracy: 0.9604
Epoch 4/1000
40/40 [==============================] - 5s 121ms/step - loss: 0.1639 - x_prob_reshape_loss: 0.1092 - x_boxes_reshape_loss: 0.0547 - x_prob_reshape_accuracy: 0.9583
Epoch 5/1000
40/40 [==============================] - 5s 122ms/step - loss: 0.1487 - x_prob_reshape_loss: 0.0968 - x_boxes_reshape_loss: 0.0520 - x_prob_reshape_accuracy: 0.9616
Epoch 6/1000
40/40 [==============================] - 5s 121ms/step - loss: 0.1451 - x_prob_reshape_loss: 0.09

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▆▅▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
x_boxes_reshape_loss,█▇▆▆▆▆▆▆▆▆▆▆▅▅▅▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▃▃▃▃▂▂▁▁▁
x_prob_reshape_accuracy,▁▂▂▂▂▄▃▄▅▅▄▅▅▆▅▆▆▆▇▇▇▇▇▇▇▇▇█▇███████████
x_prob_reshape_loss,█▇▇▇▆▅▆▅▄▄▅▄▄▃▄▃▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
epoch,999
loss,0.02926
x_boxes_reshape_loss,0.0278
x_prob_reshape_accuracy,0.99993
x_prob_reshape_loss,0.00146


wandb: Agent Starting Run: 9ttv9k0j with config:
wandb: 	batch_size: 256
wandb: 	epochs: 1000
wandb: 	learning_rate: 0.05619767101878606
wandb: 	optimizer: sgd
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/1000
40/40 [==============================] - 6s 121ms/step - loss: 0.4884 - x_prob_reshape_loss: 0.3435 - x_boxes_reshape_loss: 0.1449 - x_prob_reshape_accuracy: 0.8569
Epoch 2/1000
40/40 [==============================] - 5s 121ms/step - loss: 0.2155 - x_prob_reshape_loss: 0.1418 - x_boxes_reshape_loss: 0.0738 - x_prob_reshape_accuracy: 0.9493
Epoch 3/1000
40/40 [==============================] - 5s 122ms/step - loss: 0.1864 - x_prob_reshape_loss: 0.1268 - x_boxes_reshape_loss: 0.0596 - x_prob_reshape_accuracy: 0.9513
Epoch 4/1000
40/40 [==============================] - 5s 121ms/step - loss: 0.1650 - x_prob_reshape_loss: 0.1091 - x_boxes_reshape_loss: 0.0559 - x_prob_reshape_accuracy: 0.9577
Epoch 5/1000
40/40 [==============================] - 5s 121ms/step - loss: 0.1573 - x_prob_reshape_loss: 0.1039 - x_boxes_reshape_loss: 0.0534 - x_prob_reshape_accuracy: 0.9579
Epoch 6/1000
40/40 [==============================] - 5s 122ms/step - loss: 0.1474 - x_prob_reshape_loss: 0.09

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▆▆▆▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
x_boxes_reshape_loss,█▇▆▆▆▅▅▅▅▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▄▄▄▄▄▄▃▃▃▃▂▂▂▂▁▁
x_prob_reshape_accuracy,▁▁▃▃▃▄▄▅▅▅▅▅▆▅▆▆▇▇▇▇▇▇▇▇████████████████
x_prob_reshape_loss,█▇▆▆▆▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,999
loss,0.02975
x_boxes_reshape_loss,0.02875
x_prob_reshape_accuracy,1.0
x_prob_reshape_loss,0.001


In [13]:
import random
import string

def random_task_name(length=8):
    # Generate a random string of letters and digits
    letters_digits = string.ascii_letters + string.digits
    return ''.join(random.choice(letters_digits) for i in range(length))

task_name = random_task_name()


import wandb

# Assuming task_name is generated from one of the methods above
wandb.config = {"task_name": task_name}

# Initialize your W&B run
wandb.init(project="your_project_name", entity="your_entity_name")





wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


In [14]:
print(f"Running experiment with task: {wandb.config.task_name}")

AttributeError: <class 'wandb.sdk.wandb_config.Config'> object has no attribute 'task_name'